## Simple LLM LCEL

Groq é uma empresa que desenvolve hardware especializado para acelerar aplicações de inteligência artificial (IA), especialmente aquelas que exigem baixa latência e alta velocidade, como modelos de linguagem e visão computacional. Seu principal produto é o GroqChip, um processador projetado para executar inferência de IA de forma extremamente rápida e eficiente.

A principal vantagem da Groq está na velocidade de processamento determinística, ou seja, ela entrega resultados previsíveis e muito rápidos, ideal para aplicações em tempo real, como tradução simultânea, direção autônoma e serviços de IA conversacional. Além disso, o chip é altamente otimizado para modelos de IA já treinados, o que reduz o custo energético e melhora a eficiência.

Outra vantagem é sua simplicidade de programação, com uma arquitetura linear que facilita o desenvolvimento e a integração com sistemas existentes. A Groq também permite o uso de ferramentas populares como TensorFlow e PyTorch, tornando a migração mais simples para empresas que já usam essas plataformas.

Como desvantagem, a Groq ainda é uma tecnologia relativamente nova no mercado, com menor adoção em comparação com gigantes como NVIDIA. Isso pode significar menos suporte da comunidade e menor compatibilidade com algumas soluções específicas. Além disso, seu foco está na inferência, não no treinamento de modelos, o que limita seu uso em ambientes de desenvolvimento de IA mais completos.

#### Significado do termo Inferência

Em inteligência artificial, inferência é o processo em que um modelo treinado é usado para fazer previsões ou tomar decisões com base em novos dados. Ou seja, depois que o modelo "aprende" durante a fase de treinamento, ele entra na fase de inferência, onde aplica esse conhecimento em situações reais.

Na prática, imagine um modelo treinado para identificar gatos em imagens. Durante a inferência, você envia uma nova imagem (que o modelo nunca viu antes), e ele analisa essa imagem e responde: “Sim, isso é um gato” ou “Não, não é um gato”. O modelo não aprende mais nessa fase, apenas usa o que já aprendeu.

Esse processo precisa ser rápido e eficiente, especialmente em aplicativos como carros autônomos, assistentes virtuais ou traduções em tempo real, onde cada milissegundo conta. É por isso que empresas como a Groq desenvolvem hardware otimizado: para acelerar justamente essa etapa de inferência.

Resumindo: treinamento é onde o modelo aprende; inferência é onde ele mostra o que aprendeu ao fazer previsões em tempo real.

In [4]:
## Open AI API Key and Open Source models - Llama3, Gemma2, Mistral -- Groq

import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

In [11]:
groq_api_key = os.getenv('GROQ_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

In [14]:
model = ChatGroq(model='Gemma2-9b-it', groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7641b6b7ace0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7641b6ba9180>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [16]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to Spanish"),
    HumanMessage(content="Hello How are you?")
]

result = model.invoke(messages)


In [17]:
result

AIMessage(content='Here\'s the translation of "Hello, how are you?" to Spanish:\n\n**Formal:**\n\n* **Hola, ¿cómo está?** (used when addressing someone you don\'t know well or someone older than you)\n\n**Informal:**\n\n* **Hola, ¿cómo estás?** (used with friends, family, and people your own age)\n\n\nLet me know if you have any other phrases you\'d like translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 21, 'total_tokens': 114, 'completion_time': 0.169090909, 'prompt_time': 0.002144385, 'queue_time': 0.15899489, 'total_time': 0.171235294}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--bf45fe0b-2636-41d0-86e9-caf134633cca-0', usage_metadata={'input_tokens': 21, 'output_tokens': 93, 'total_tokens': 114})

In [18]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'Here\'s the translation of "Hello, how are you?" to Spanish:\n\n**Formal:**\n\n* **Hola, ¿cómo está?** (used when addressing someone you don\'t know well or someone older than you)\n\n**Informal:**\n\n* **Hola, ¿cómo estás?** (used with friends, family, and people your own age)\n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [19]:
### Using LCEL - chain the components

chain = model | parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in Spanish:\n\n* **Hola, ¿cómo estás?** (informal, to one person)\n* **Hola, ¿cómo está?** (formal, to one person)\n* **Hola, ¿cómo están?** (formal, to multiple people)\n\n\n Let me know if you\'d like to learn other Spanish greetings! \n'

In [22]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_templates = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [('system', generic_templates), ('user', '{text}')]
)

In [23]:
result = prompt.invoke({'language': 'Spanish', 'text': 'Hello'})

In [24]:
result.to_messages()

[SystemMessage(content='Translate the following into Spanish:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [26]:
chain = prompt | model | parser
chain.invoke({'language': "Spanish", "text": "Hello, goode morning."})

'Hola, buenos días. \n'